# Observed value & Expected value

## 빈도의 함정

데이터를 관찰 할 때, 우리는 주로 얼마나 다른가? 얼마나 특이한가?를 살펴보게 된다. 

예를 들어 어떤 과학자 이름을 데이터로 삼아 분석을 하고 나서 이 데이터의 특징 가운데 하나로 김씨, 이씨, 박씨가 가장 많았다고 결론을 내렸다고 하자. 텍스트 분석은 빈도를 기반으로 하고, 빈도 상에서 분명 김씨, 이씨, 박씨가 많았으므로 해당 데이터의 성격이라고 할 수 있을 것이다. 그러나 한국 이름에서 이들 성씨가 가장 많다는 점을 안다면, 해당 데이터만의 중요한 특징이라고 주장하기는 어려울 것이다. 

또 다른 예를 보자. 해수를 치료하는 처방 데이터를 모두 보았다고 해 보자. 그리고 나서 해수와 가장 관련이 높은 본초를 이야기 하기 위해 빈도를 조사했다고 하자. 조사 결과는 아마도 감초, 대조, 생강 순으로 나왔을 것이다. 이들 본초는 모든 병증에서 두루 많이 사용되고 있기 때문에 해수의 특성을 나타내는 본초라고 보기 힘들다. 

이상의 예를 보면, 단순히 빈도만을 가지고 특성을 설명하면 사실이라고 할지라도 원하는 목적을 이루지 못할 가능성이 크다. Document-Term Matrix에서 TF matrix보다 TFIDF matrix가 더 일반적으로 사용되는 것도 이러한 문제를 극복하기 위한 것이다. 

## 관찰빈도의 기대빈도

빈도의 함정을 해결하기 위해 사용하는 가장 단순한 방법은 정규화(normalize)이다. 데이터를 동일한 scale로 맞추어 비교를 용이하게 하는 방법이다. 하지만 명목변수가 많은 텍스트 분석에는 적용하기 어려울 때가 많다. 

텍스트 분석에서는 대개 해당 텍스트가 명목변수 역할을 하기 때문에 문제의 양상이 다르다. 따라서 그에 맞는 방법을 사용해야 한다. 바로 관찰빈도와 기대빈도의 차이를 살펴보는 것이다. 

관찰빈도는 실제 관찰된 빈도이다. 즉, 내가 가지고 있는 데이터 그대로의 값이다. 기대빈도는 가상의 추정 값으로, 두 변수가 독립이라는 가정 하에 얼마나 나타날 것인지를 추정한 값이다. 

예를 들어, 전체 corpus에 처방이 1000개 있고, 전체 처방 가운데 해수를 치료하는 처방이 80개, 감초가 들어간 처방이 500개라고 해보자. 해수 처방만 보았을 때 감초가 포함된 처방이 20개이다. 해수와 감초의 관계를 본다고 하였을 때, 기대빈도는 관찰된 빈도인 20이다. 해수 처방일 확률이 80/1000, 감초가 포함된 처방일 확률이 500/1000이다. 두 확률이 서로 관계가 없다면, 해수를 치료하면서 감초가 포함된 처방은 1000 × (80/1000) × (500/1000)으로 40이 된다. 따라서 기대빈도는 40이다. 관찰된 값이 기대되는 값보다 작으므로 해수를 치료하는 처방에서는 오히려 감초가 적게 사용되었다고 말해야 한다. 


## Contingency table (confusion_matrix)

위 예시에서 언급된 내용을 contingency table을 만들어 보자. 

|  빈도  | 감초O | 감초X | margin |
|:------:|:-----:|:-----:|:------:|
|  해수O |   20  |   -   |   80   |
|  해수X |   -   |   -   |    -   |
| margin |  500  |   -   |  1000  |

빈 칸의 값들은 주어진 값들로 부터 도출해 낼 수 있다. 

|  빈도  | 감초O | 감초X | margin |
|:------:|:-----:|:-----:|:------:|
|  해수O |   20  |   60  |   80   |
|  해수X |  480  |  440  |   920  |
| margin |  500  |  500  |  1000  |

이상이 관찰빈도에 대한 contingency table이다. 

우리는 여기서 감초와 해수가 서로 관계가 없다고 가정하고, 새로운 contingency table을 만들 수 있다. 

|  빈도  | 감초O | 감초X | margin |
|:------:|:-----:|:-----:|:------:|
|  해수O |   -   |   -   |   80   |
|  해수X |   -   |   -   |   920  |
| margin |  500  |  500  |  1000  |

여기에서도 빈 칸의 값들을 margin 값을 동일한 비율이 되도록 채워 나갈 수 있다. 

|  빈도  | 감초O | 감초X | margin |
|:------:|:-----:|:-----:|:------:|
|  해수O |   40  |   40  |   80   |
|  해수X |  460  |  460  |   920  |
| margin |  500  |  500  |  1000  |

이상이 기대빈도에 대한  contingency table이다. 


In [25]:
import numpy as np

observed_value = np.array( [
    [  20, 60  ],
    [ 480, 440 ] ])

margin_row = np.sum( observed_value, 1 )  # array([ 80, 920])
margin_col = np.sum( observed_value, 0 )  # array([500, 500])
sum_all = np.sum( margin_row )            # 1000

print( observed_value )

[[ 20  60]
 [480 440]]


In [26]:
def expected( o_v ):
    margin_row = np.sum( o_v, 1 ) 
    margin_col = np.sum( o_v, 0 ) 
    sum_all = np.sum( margin_row ) 
    e_v = np.zeros_like( o_v )
    e_v[0][0] = sum_all * ( margin_row[0]/sum_all ) * ( margin_col[0]/sum_all )
    e_v[0][1] = sum_all * ( margin_row[0]/sum_all ) * ( margin_col[1]/sum_all )
    e_v[1][0] = sum_all * ( margin_row[1]/sum_all ) * ( margin_col[0]/sum_all )
    e_v[1][1] = sum_all * ( margin_row[1]/sum_all ) * ( margin_col[1]/sum_all )
    return e_v

expected_value = expected( observed_value )
print( expected_value )

[[ 40  40]
 [460 460]]
